In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
x_train, x_test = x_train/255.0, x_test/255.0

In [6]:
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [7]:
mlp_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy',
                  metrics=['accuracy'])

In [8]:
mlp_model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2548 - accuracy: 0.9284
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1122 - accuracy: 0.9659
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0765 - accuracy: 0.9770
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0576 - accuracy: 0.9830
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0443 - accuracy: 0.9862


In [10]:
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)

In [11]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmphkg09f_n\assets


INFO:tensorflow:Assets written to: C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmphkg09f_n\assets


In [43]:
with open('./mnist.tflite', 'wb') as f:
    f.write(tflite_model)

In [46]:
mlp_model.save("./mlp_model2/")

ValueError: Model <__main__.MLP_Model object at 0x000001C482559880> cannot be saved either because the input shape is not available or because the forward pass of the model is not defined.To define a forward pass, please override `Model.call()`. To specify an input shape, either call `build(input_shape)` directly, or call the model on actual data using `Model()`, `Model.fit()`, or `Model.predict()`. If you have a custom training step, please make sure to invoke the forward pass in train step through `Model.__call__`, i.e. `model(inputs)`, as opposed to `model.call()`.

In [45]:
saved_model = tf.keras.models.load_model("./mlp_model2/")

In [18]:
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model2/")

In [19]:
tflite_model = converter.convert()

In [20]:
with open('./saved_model.tflite', 'wb') as f:
    f.write(tflite_model)

In [21]:
class Inc(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs + 1

In [22]:
inc = Inc()

In [23]:
class Inc_Graph(tf.keras.layers.Layer):
    @tf.function
    def call(self, inputs):
        return inputs + 1
inc_g = Inc_Graph()

In [24]:
inc_g2 = tf.function(inc)

In [25]:
print(inc_g(tf.constant(3)))
print(inc_g(tf.constant([3,2])))
print(inc_g(tf.constant([[3,2],[1.0,5.0]])))

print(inc_g2(tf.constant(3)))
print(inc_g2(tf.constant([3,2])))
print(inc_g2(tf.constant([[3,2],[1.0,5.0]])))

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)
tf.Tensor(
[[4. 3.]
 [2. 6.]], shape=(2, 2), dtype=float32)


In [26]:
print(inc_g(tf.constant(4)))
print(inc_g2(tf.constant(4)))

tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


In [27]:
concrete_fun = inc_g.call.get_concrete_function(tf.TensorSpec(shape=(1,3),
                                                              dtype=tf.float32))
print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))

tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)


In [30]:
concrete_fun = inc_g2.get_concrete_function(tf.TensorSpec(shape=(1, 3), dtype=tf.float32))
print(concrete_fun(tf.constant([[1.0, 2.0, 3.0]])))


tf.Tensor([[2. 3. 4.]], shape=(1, 3), dtype=float32)


In [31]:
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [32]:
graph_model = tf.function(mlp_model)
concerte_func = graph_model.get_concrete_function(
    tf.TensorSpec(shape=mlp_model.inputs[0].shape, dtype=mlp_model.inputs[0].dtype))

In [38]:
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(128, activation='relu')
        self.softmax = tf.keras.layers.Dense(10, activation='softmax')
        
    @tf.function
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense(x)
        return self.softmax(x)